### SegNet For Denoising 2

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
learning_rate = 1e-3
batch_size = 32
training_epochs = 300

In [3]:
filename = 'splitted_dataset_2.npz'
dataset = np.load(filename)

In [4]:
inputs_train = dataset['inputs_train_1']
cleans_train = dataset['cleans_train_1']
inputs_test  = dataset['inputs_test_1']
cleans_test  = dataset['cleans_test_1']

In [5]:
print(inputs_train.shape)
print(cleans_train.shape)
print(inputs_test.shape)
print(cleans_test.shape)

(1800, 2, 257, 382)
(1800, 2, 257, 382)
(200, 2, 257, 382)
(200, 2, 257, 382)


In [6]:
class AutoEncoder_dataset(Dataset):
    def __init__(self, inputs, cleans):
        self.inputs = torch.from_numpy(inputs).float()
        self.cleans = torch.from_numpy(cleans).float()
    
    def __getitem__(self, index):
        inputs = self.inputs[index]
        cleans = self.cleans[index]
        return inputs, cleans
    
    def __len__(self):
        return len(self.inputs)

In [7]:
train_set = AutoEncoder_dataset(inputs = inputs_train,
                                cleans = cleans_train)
test_set  = AutoEncoder_dataset(inputs = inputs_test,
                                cleans = cleans_test)

In [8]:
train_loader = DataLoader(dataset=train_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
test_loader = DataLoader(dataset=test_set,
                         batch_size=batch_size,
                         shuffle=False,
                         drop_last=False)

In [9]:
class SegNet(nn.Module):
    
    def __init__(self):
        super(SegNet, self).__init__()
        
        # Encoder
        
        self.Enc_0_1 = nn.Sequential(
            nn.Conv2d(in_channels=2, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True)
        )
        
        self.Enc_1_1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(inplace=True)
        )
        
        self.Enc_2_1 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels= 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True)
        )
        
        self.Enc_2_2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels= 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True)
        )
        
        self.Enc_3_1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels= 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        
        self.Enc_3_2 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels= 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        
        self.Enc_4_1 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels= 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        
        self.Enc_4_2 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels= 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        
        self.Pool = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        
        # Decoder
        
        self.Dec_4_2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        
        self.Dec_4_1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        
        self.Dec_3_2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(inplace=True)
        )
        
        self.Dec_3_1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True)
        )
        
        self.Dec_2_2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True)
        )
        
        self.Dec_2_1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(inplace=True)
        )
        
        self.Dec_1_1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True)
        )
        
        self.Dec_0_1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=64, out_channels=2, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=2),
            nn.ReLU(inplace=True)
        )
        
        self.Unpool = nn.MaxUnpool2d(kernel_size=2, stride=2)
        
        
    def forward(self, inputs):
        
        # Encoder
        
        dim_0 = inputs.size()
        outputs = self.Enc_0_1(inputs)
        outputs, indice_0 = self.Pool(outputs)
        
        dim_1 = outputs.size()
        outputs = self.Enc_1_1(outputs)
        outputs, indice_1 = self.Pool(outputs)
        
        dim_2 = outputs.size()
        outputs = self.Enc_2_1(outputs)
        outputs = self.Enc_2_2(outputs)
        outputs, indice_2 = self.Pool(outputs)
        
        dim_3 = outputs.size()
        outputs = self.Enc_3_1(outputs)
        outputs = self.Enc_3_2(outputs)
        outputs, indice_3 = self.Pool(outputs)
        
        dim_4 = outputs.size()
        outputs = self.Enc_4_1(outputs)
        outputs = self.Enc_4_2(outputs)
        outputs, indice_4 = self.Pool(outputs)
        
        dim_middle = outputs.size()
        
        # Decoder
        outputs = self.Unpool(outputs, indice_4, output_size=dim_4)
        outputs = self.Dec_4_2(outputs)
        outputs = self.Dec_4_1(outputs)
        dim_4d = outputs.size()
        
        outputs = self.Unpool(outputs, indice_3, output_size=dim_3)
        outputs = self.Dec_3_2(outputs)
        outputs = self.Dec_3_1(outputs)
        dim_3d = outputs.size()
        
        outputs = self.Unpool(outputs, indice_2, output_size=dim_2)
        outputs = self.Dec_2_2(outputs)
        outputs = self.Dec_2_1(outputs)
        dim_2d = outputs.size()
        
        outputs = self.Unpool(outputs, indice_1, output_size=dim_1)
        outputs = self.Dec_1_1(outputs)
        dim_1d = outputs.size()
        
        outputs = self.Unpool(outputs, indice_0, output_size=dim_0)
        outputs = self.Dec_0_1(outputs)
        dim_0d = outputs.size()
        
        return outputs
        

In [10]:
GPU_NUM = 7
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Currernt cuda device ', torch.cuda.current_device())

if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Currernt cuda device  7
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [11]:
net = SegNet().to(device)

In [12]:
criterion = nn.MSELoss()

In [13]:
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [14]:
def save_checkpoint(epoch, model, optimizer, filename):
    state = {
        'Epoch': epoch,
        'State_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, filename)

In [15]:
total_batch = len(train_loader)


for epoch in range(training_epochs):
    avg_cost = 0

    for i, (inputs, cleans) in enumerate(train_loader):
        inputs = inputs.to(device)
        cleans = cleans.to(device)

        optimizer.zero_grad()
        hypothesis = net(inputs)
        cost = criterion(hypothesis, cleans)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    if epoch % 10 == 9:
        print('[Epoch : {:>4} / {:>3}] cost = {:>.9}'.format(epoch + 1, training_epochs, avg_cost))
        
        save_checkpoint(epoch, net, optimizer, 'SegNet.pt')
        print("======= Saved Model =======")
        
print('Learning finished')

[Epoch :   10 / 300] cost = 9.08768707e-05
======= Saved Model =======
[Epoch :   20 / 300] cost = 8.69568175e-05
======= Saved Model =======
[Epoch :   30 / 300] cost = 8.68895149e-05
======= Saved Model =======
[Epoch :   40 / 300] cost = 8.61078952e-05
======= Saved Model =======
[Epoch :   50 / 300] cost = 8.61536755e-05
======= Saved Model =======
[Epoch :   60 / 300] cost = 8.59345309e-05
======= Saved Model =======
[Epoch :   70 / 300] cost = 8.58029816e-05
======= Saved Model =======
[Epoch :   80 / 300] cost = 8.32959413e-05
======= Saved Model =======
[Epoch :   90 / 300] cost = 4.06738945e-05
======= Saved Model =======
[Epoch :  100 / 300] cost = 3.265536e-05
======= Saved Model =======
[Epoch :  110 / 300] cost = 3.137155e-05
======= Saved Model =======
[Epoch :  120 / 300] cost = 2.51261536e-05
======= Saved Model =======
[Epoch :  130 / 300] cost = 8.66508053e-05
======= Saved Model =======
[Epoch :  140 / 300] cost = 4.02044934e-05
======= Saved Model =======
[Epoch :  

In [16]:
total_batch = len(train_loader)


for epoch in range(training_epochs):
    avg_cost = 0

    for i, (inputs, cleans) in enumerate(train_loader):
        inputs = inputs.to(device)
        cleans = cleans.to(device)

        optimizer.zero_grad()
        hypothesis = net(inputs)
        cost = criterion(hypothesis, cleans)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    if epoch % 10 == 9:
        print('[Epoch : {:>4} / {:>3}] cost = {:>.9}'.format(epoch + 1 + 300, training_epochs + 300, avg_cost))
        
        save_checkpoint(epoch, net, optimizer, 'SegNet.pt')
        print("======= Saved Model =======")
        
print('Learning finished')

[Epoch :  310 / 600] cost = 7.92656738e-06
======= Saved Model =======
[Epoch :  320 / 600] cost = 7.68245172e-06
======= Saved Model =======
[Epoch :  330 / 600] cost = 7.30007196e-06
======= Saved Model =======
[Epoch :  340 / 600] cost = 7.18075762e-06
======= Saved Model =======
[Epoch :  350 / 600] cost = 6.58737827e-06
======= Saved Model =======
[Epoch :  360 / 600] cost = 6.51382925e-06
======= Saved Model =======
[Epoch :  370 / 600] cost = 6.5266081e-06
======= Saved Model =======
[Epoch :  380 / 600] cost = 5.90251193e-06
======= Saved Model =======
[Epoch :  390 / 600] cost = 6.07209404e-06
======= Saved Model =======
[Epoch :  400 / 600] cost = 5.67085726e-06
======= Saved Model =======
[Epoch :  410 / 600] cost = 5.75515742e-06
======= Saved Model =======
[Epoch :  420 / 600] cost = 5.45606599e-06
======= Saved Model =======
[Epoch :  430 / 600] cost = 5.22341634e-06
======= Saved Model =======
[Epoch :  440 / 600] cost = 4.8399811e-06
======= Saved Model =======
[Epoch :

In [17]:
total_batch = len(train_loader)


for epoch in range(training_epochs):
    avg_cost = 0

    for i, (inputs, cleans) in enumerate(train_loader):
        inputs = inputs.to(device)
        cleans = cleans.to(device)

        optimizer.zero_grad()
        hypothesis = net(inputs)
        cost = criterion(hypothesis, cleans)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    if epoch % 10 == 9:
        print('[Epoch : {:>4} / {:>3}] cost = {:>.9}'.format(epoch + 1 + 600, training_epochs + 600, avg_cost))
        
        save_checkpoint(epoch, net, optimizer, 'SegNet.pt')
        print("======= Saved Model =======")
        
print('Learning finished')

[Epoch :  610 / 900] cost = 3.62000992e-06
======= Saved Model =======
[Epoch :  620 / 900] cost = 3.39115286e-06
======= Saved Model =======
[Epoch :  630 / 900] cost = 3.57259933e-06
======= Saved Model =======
[Epoch :  640 / 900] cost = 3.43366332e-06
======= Saved Model =======
[Epoch :  650 / 900] cost = 3.27461976e-06
======= Saved Model =======
[Epoch :  660 / 900] cost = 3.28709484e-06
======= Saved Model =======
[Epoch :  670 / 900] cost = 3.32075092e-06
======= Saved Model =======
[Epoch :  680 / 900] cost = 3.10164955e-06
======= Saved Model =======
[Epoch :  690 / 900] cost = 3.09115467e-06
======= Saved Model =======
[Epoch :  700 / 900] cost = 3.05063372e-06
======= Saved Model =======
[Epoch :  710 / 900] cost = 2.99575504e-06
======= Saved Model =======
[Epoch :  720 / 900] cost = 3.09315897e-06
======= Saved Model =======
[Epoch :  730 / 900] cost = 3.01417913e-06
======= Saved Model =======
[Epoch :  740 / 900] cost = 2.93542917e-06
======= Saved Model =======
[Epoch

In [18]:
total_batch = len(train_loader)


for epoch in range(training_epochs):
    avg_cost = 0

    for i, (inputs, cleans) in enumerate(train_loader):
        inputs = inputs.to(device)
        cleans = cleans.to(device)

        optimizer.zero_grad()
        hypothesis = net(inputs)
        cost = criterion(hypothesis, cleans)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    if epoch % 10 == 9:
        print('[Epoch : {:>4} / {:>3}] cost = {:>.9}'.format(epoch + 1 + 900, training_epochs + 900, avg_cost))
        
        save_checkpoint(epoch, net, optimizer, 'SegNet2.pt')
        print("======= Saved Model =======")
        
print('Learning finished')

[Epoch :  910 / 1200] cost = 2.5854747e-06
======= Saved Model =======
[Epoch :  920 / 1200] cost = 2.67442238e-06
======= Saved Model =======
[Epoch :  930 / 1200] cost = 2.42542092e-06
======= Saved Model =======
[Epoch :  940 / 1200] cost = 2.73721412e-06
======= Saved Model =======
[Epoch :  950 / 1200] cost = 2.45288334e-06
======= Saved Model =======
[Epoch :  960 / 1200] cost = 2.44686544e-06
======= Saved Model =======
[Epoch :  970 / 1200] cost = 2.44471516e-06
======= Saved Model =======
[Epoch :  980 / 1200] cost = 2.4071212e-06
======= Saved Model =======
[Epoch :  990 / 1200] cost = 2.35924267e-06
======= Saved Model =======
[Epoch : 1000 / 1200] cost = 2.39693395e-06
======= Saved Model =======
[Epoch : 1010 / 1200] cost = 2.29529178e-06
======= Saved Model =======
[Epoch : 1020 / 1200] cost = 2.24501537e-06
======= Saved Model =======
[Epoch : 1030 / 1200] cost = 2.27298028e-06
======= Saved Model =======
[Epoch : 1040 / 1200] cost = 2.30951696e-06
======= Saved Model ==